# Kaggle Competition Site - Kenyan Food Image Classification


This competition is about classification of Kenyan food images.
The dataset consists of 8,174 images in 13 Kenyan food type classes. 

The details of the competition:
https://www.kaggle.com/c/opencv-pytorch-dl-course-classification

## Summary

Trained 50 epochs using modified ResNet152 network

Modified the original single fully connected layer to 2 fully connected layers with 2 dropout layers

Unfreezed Layer 4 in the original ResNet152 network and the 2 newly added fully connected layers. Freezed all other layers (Transfer Learning)

Used the model weights at Epoch 31 (Early Stopping)

Training Accuracy:   81.9%

Validation Accuracy: 78.3%

Test Accuracy:       77.5%


## TensorBoard Dev Scalars Log Link 

https://tensorboard.dev/experiment/0NHYnHGqS2OuOouVrxKejw/

## Competition Leaderboard

https://www.kaggle.com/c/opencv-pytorch-dl-course-classification/leaderboard

## Code

In [ ]:
!rm -rf /kaggle/working/trainer
!mkdir -p /kaggle/working/images
!cp /kaggle/input/opencv-pytorch-dl-course-classification/images/images/* /kaggle/working/images
!cp /kaggle/input/opencv-pytorch-dl-course-classification/*.csv /kaggle/working
!mkdir -p /kaggle/working/dataset
!cd /kaggle/working

In [ ]:
import os
import shutil
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from operator import itemgetter
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader

from PIL import Image
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

import random
import datetime
from typing import Callable, Iterable
import torch.nn.functional as F
from dataclasses import dataclass
from torchvision.datasets import ImageFolder
from torchvision.transforms import functional as TF
from torch.utils.tensorboard import SummaryWriter
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [ ]:
def seed_everything(sys_config):
    random.seed(sys_config.seed)
    np.random.seed(sys_config.seed)
    torch.manual_seed(sys_config.seed)
    torch.backends.cudnn.deterministic = sys_config.cudnn_deterministic
    torch.backends.cudnn.benchmark = sys_config.cudnn_benchmark_enabled

### Define System and Training Configurations

In [ ]:
@dataclass
class SystemConfiguration:
    '''
    Describes the common system setting needed for reproducible training
    '''
    seed: int = 42                          # seed number to set the state of all random number generators
    cudnn_benchmark_enabled: bool = True    # enable CuDNN benchmark for the sake of performance
    cudnn_deterministic: bool = True        # make cudnn deterministic (reproducible training)

@dataclass
class TrainerConfig:
    """
    Describes configuration for the training process
    """
    batch_size: int = 16
    resnet_model_name = "resnet152"        # change network structure here
    
    num_epochs: int = 50
    learning_rate: float = 0.00001
    
    data_root: str = r"/kaggle/working"
    root_log_dir: str = r"runs"
    root_checkpoint_dir: str = r"checkpoints"
    num_workers: int = max(2, os.cpu_count() - 2)
    device: str = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    save_freq: str = "each"                 # "best" OR "each"

### Split Data

In [ ]:
def generateDatasetStructure(class_names, data_root, train_ratio=0.75):
    # write class labels to file
    dataset_dir = os.path.join(data_root, 'dataset')
    os.makedirs(dataset_dir, exist_ok=True)
    file = open(os.path.join(dataset_dir, 'class_names.txt'), 'w')
    for class_name in class_names:
        file.write(class_name + '\n')
    file.close()

    # creating directories
    training_dir = os.path.join(data_root, 'dataset/training')
    validation_dir = os.path.join(data_root, 'dataset/validation')
    os.makedirs(training_dir, exist_ok=True)
    os.makedirs(validation_dir, exist_ok=True)
    for class_name in class_names:
        os.makedirs(os.path.join(training_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(validation_dir, class_name), exist_ok=True)

    # read image names from 'train.csv' into a hashmap: class_name -> image_names
    train_csv_path = os.path.join(data_root, 'train.csv')
    raw_data_info = pd.read_csv(train_csv_path, engine='python', dtype={0:str, 1:str})
    class_to_img_path = {class_name:[] for class_name in class_names}

    for filename, class_name in raw_data_info.values:
        full_file_path = os.path.join(data_root, 'images', filename + '.jpg')
        class_to_img_path[class_name].append(full_file_path)

    # split images into training/validation and put images in each class folders 
    for class_name in class_to_img_path:
        data_len_single_class = len(class_to_img_path[class_name])
        training_src_paths = class_to_img_path[class_name][:int(train_ratio * data_len_single_class)]
        validation_src_paths = class_to_img_path[class_name][int(train_ratio * data_len_single_class):]
        training_dst_path = os.path.join(training_dir, class_name)
        validation_dst_path = os.path.join(validation_dir, class_name)
        for path in training_src_paths:
            shutil.copy(path, training_dst_path)
        for path in validation_src_paths:
            shutil.copy(path, validation_dst_path)
            
    return class_to_img_path

### Transforms and Augmentation

In [ ]:
def image_preprocess_transforms():
    preprocess = transforms.Compose([
    transforms.Resize(350),
    transforms.CenterCrop(300),
    transforms.ToTensor()
    ])
    
    return preprocess

def image_common_transforms(mean=(0.4611, 0.4359, 0.3905), std=(0.2193, 0.2150, 0.2109)):
    preprocess = image_preprocess_transforms()
    
    common_transforms = transforms.Compose([
        preprocess,
        transforms.Normalize(mean, std)
    ])
    
    return common_transforms

def data_augmentation_preprocess(common_transforms):
    
    data_augmented_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(20, fill=(0,0,0)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        common_transforms
    ])
    
    return data_augmented_transforms

### Calculate Image mean and std

In [ ]:
def get_mean_std(data_root, batch_size=16, num_workers=4):
    preprocess = image_preprocess_transforms()
    
    training_set_with_basic_transforms = ImageFolder(
        os.path.join(data_root, "dataset", "training"), transform=preprocess
    )

    loader = torch.utils.data.DataLoader(
        training_set_with_basic_transforms,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
    )

    mean = 0.0
    std = 0.0

    for images, _ in loader:
        batch_samples = images.size(0)
        images = images.view(batch_samples, images.size(1), -1)
        mean += images.mean(2).sum(0)
        std += images.std(2).sum(0)
    mean /= len(loader.dataset)
    std /= len(loader.dataset)

    print("mean: {}, std: {}".format(mean, std))

    return mean, std

### Create Dataset and Dataloader

In [ ]:
def get_data(config, data_augmentation=False):

    mean, std = get_mean_std(config.data_root, batch_size=config.batch_size, num_workers=config.num_workers)
    common_transforms = image_common_transforms(mean, std)

    # data augmentation implementation
    if data_augmentation:
        train_transforms = transforms.Compose(
            [
                data_augmentation_preprocess(common_transforms),
                transforms.RandomErasing(),
            ]
        )
    else:
        train_transforms = common_transforms
        
    training_set = ImageFolder(
        os.path.join(config.data_root, "dataset", "training"), transform=train_transforms
    )

    validation_set = ImageFolder(
        os.path.join(config.data_root, "dataset", "validation"), transform=common_transforms
    )

    pin = True if config.device == torch.device("cuda") else False
    
    training_loader = torch.utils.data.DataLoader(
        training_set,
        batch_size=config.batch_size,
        shuffle=True,
        num_workers=config.num_workers,
        pin_memory=pin,
    )
    
    validation_loader = torch.utils.data.DataLoader(
        validation_set,
        batch_size=config.batch_size,
        shuffle=False,
        num_workers=config.num_workers,
        pin_memory=pin,
    )

    return training_loader, validation_loader

### ResNet Model Creation

In [ ]:
def pretrained_resnet(model_name, transfer_learning=True, num_class=13):          # adjust the number of classes here
    if model_name == "resnet18":
        resnet = models.resnet18(pretrained=True)
    elif model_name == "resnet50":
        resnet = models.resnet50(pretrained=True)
    elif model_name == "resnet101":
        resnet = models.resnet101(pretrained=True)
    elif model_name == "resnet152":
        resnet = models.resnet152(pretrained=True)
    elif model_name == "resnext50_32x4d":
        resnet = models.resnext50_32x4d(pretrained=True)
    
    if transfer_learning:
        for param in resnet.parameters():
            param.requires_grad = False

        resnet.layer4.requires_grad_(True)          
    
    last_layer_in = resnet.fc.in_features
    
    resnet.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(last_layer_in, 256),
        nn.Dropout(0.5),
        nn.Linear(256, num_class)
    )
    
    return resnet

### Training/Validation Utility functions

In [ ]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [ ]:
def train_one_epoch(
    model,
    optimizer,
    device,
    epoch,
    num_epochs,
    *,
    loss_fn,
    dataloader,
    writer=None,
    lr_scheduler=None
):
    avg_train_acc = 0.0
    avg_train_loss = 0.0
    p_avg_train_loss = 0.0
    p_avg_train_acc = 0.0

    prefix = f"{epoch+1:03}/{num_epochs:03}"
    length_loader = len(dataloader)
    iteration = length_loader * epoch

    for idx, batch in enumerate(dataloader, 1):
        data = batch[0].to(device)
        target = batch[1].to(device)

        output = model(data)

        batch_loss = loss_fn(output, target)

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
        batch_acc = accuracy(output.detach(), target.detach()).item()

        if lr_scheduler:
            lr_scheduler.step()
        batch_loss = batch_loss.detach().item()

        # calculate running average
        avg_train_loss = (avg_train_loss * (idx - 1) + batch_loss) / idx
        avg_train_acc = (avg_train_acc * (idx - 1) + batch_acc) / idx

        p_avg_train_loss = round(avg_train_loss, 3)
        p_avg_train_acc = round(avg_train_acc, 3)

        writer.add_scalar("Train/batch_loss", batch_loss, global_step=iteration + idx)
        writer.add_scalar("Train/batch_acc", batch_acc, global_step=iteration + idx)

        status = f"\rEpoch: {prefix} Iteration: {idx:03}/{length_loader:03} [Train] Accuracy: {p_avg_train_acc} Loss: {p_avg_train_loss} batch_acc: {round(batch_acc, 3)}  batch_loss: {round(batch_loss, 3)} LR: {optimizer.param_groups[0]['lr']}"
        print(status)

    writer.add_scalar("Train/Loss", p_avg_train_loss, global_step=epoch)
    writer.add_scalar("Train/Acc", p_avg_train_acc, global_step=epoch)

    return p_avg_train_acc, p_avg_train_loss

In [ ]:
def validation_epoch(
    model,
    device,
    epoch,
    num_epochs,
    *,
    loss_fn,
    dataloader,
    writer=None,
):

    avg_valid_acc = 0.0
    avg_valid_loss = 0.0

    p_avg_valid_loss = 0.0
    p_avg_valid_acc = 0.0

    prefix = f"{epoch+1:03}/{num_epochs:03}"
    length_loader = len(dataloader)
    iteration = length_loader * epoch

    for idx, batch in enumerate(dataloader, 1):
        data = batch[0].to(device)
        target = batch[1].to(device)

        with torch.no_grad():
            output = model(data)

        batch_loss = loss_fn(output, target).detach().item()
        batch_acc = accuracy(output.detach(), target.detach()).item()

        # calculate running average
        avg_valid_loss = (avg_valid_loss * (idx - 1) + batch_loss) / idx
        avg_valid_acc = (avg_valid_acc * (idx - 1) + batch_acc) / idx

        p_avg_valid_loss = round(avg_valid_loss, 3)
        p_avg_valid_acc = round(avg_valid_acc, 3)

        status = f"\rEpoch: {prefix} Iteration: {idx:03}/{length_loader:03} [Valid] Accuracy: {p_avg_valid_acc} Loss: {p_avg_valid_loss} batch_acc: {round(batch_acc, 3)}  batch_loss: {round(batch_loss, 3)}"
        print(status)
        gc.collect()

    writer.add_scalar("Valid/Loss", p_avg_valid_loss, global_step=epoch)
    writer.add_scalar("Valid/Acc", p_avg_valid_acc, global_step=epoch)

    return p_avg_valid_acc, p_avg_valid_loss

### Training/Validation Setup

In [ ]:
class_names = [
        "bhaji",
        "chapati",
        "nyamachoma",
        "mandazi",
        "masalachips",
        "kachumbari",
        "ugali",
        "pilau",
        "matoke",
        "githeri",
        "mukimo",
        "sukumawiki",
        "kukuchoma",
    ]

augmentation = True

seed_everything(SystemConfiguration())

config = TrainerConfig()


generateDatasetStructure(class_names, config.data_root)

train_loader, val_loader = get_data(config, data_augmentation=augmentation)

model = pretrained_resnet(model_name=config.resnet_model_name, transfer_learning=True) # True: only unfreeze certain layers
                                                                                       # False: unfreeze all layers
model.to(config.device)

optimizer = torch.optim.Adam(
    model.parameters(), lr=config.learning_rate, amsgrad=True, weight_decay=1e-5
)

loss_fn = nn.CrossEntropyLoss()

lr_scheduler = None

### Tensorboard Log Setup

In [ ]:
version_number = 0

os.makedirs(config.root_log_dir, exist_ok=True)
os.makedirs(config.root_checkpoint_dir, exist_ok=True)

folders = os.listdir(config.root_log_dir)

if len(folders):
    last_version_number = int(sorted(folders)[-1].replace("version_", ""))
    version_number = last_version_number + 1
    

config.log_dir = os.path.join(config.root_log_dir, f"version_{version_number}")
config.checkpoint_dir = os.path.join(config.root_checkpoint_dir, f"version_{version_number}")

os.makedirs(config.log_dir, exist_ok=True)
os.makedirs(config.checkpoint_dir, exist_ok=True)

print(f"Logging at: {config.log_dir}")

### Main Function

In [ ]:
summary_writer = SummaryWriter(log_dir=config.log_dir)

best_val_acc = 0.0
best_epoch = 0

for epoch in range(config.num_epochs):
    model.train()
    train_acc, train_loss = train_one_epoch(
        model,
        optimizer,
        config.device,
        epoch,
        config.num_epochs,
        loss_fn=loss_fn,
        dataloader=train_loader,
        writer=summary_writer,
        lr_scheduler=lr_scheduler
    )
    print()

    model.eval()
    valid_acc, valid_loss = validation_epoch(
        model,
        config.device,
        epoch,
        config.num_epochs,
        loss_fn=loss_fn,
        dataloader=val_loader,
        writer=summary_writer,
    )
    print()

    # save model weights based on validation accuracy
    if config.save_freq == "best":
        if valid_acc > best_val_acc:
            print("Saving Model Weights.")
            best_val_acc = valid_acc
            best_epoch = epoch
            torch.save(
                model.state_dict(),
                os.path.join(config.checkpoint_dir, "best_model_") + str(datetime.datetime.now().strftime("%Y-%m-%d-%H_%M_%S"))
            )
    elif config.save_freq == "each":
        torch.save(
            model.state_dict(),
            os.path.join(
                config.checkpoint_dir, f"model_epoch-{epoch:03}-valid_acc-{valid_acc}_"
            ) + str(datetime.datetime.now().strftime("%Y-%m-%d-%H_%M_%S"))
        )
        
summary_writer.close()